# TransformChain

In [1]:
import openai
import os 
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
import re

openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base= os.getenv('OPENAI_API_BASE')
openai.api_type= "azure"
openai.api_version = os.getenv('OPENAI_API_VERSION')
deployment:str=os.getenv('CHATGPT_MODEL')

llm = AzureChatOpenAI(
    deployment_name=deployment,
    model_name=deployment
    )


In [2]:
with open("nda.txt", 'r', encoding='utf-8') as file:
    nda = file.read()


In [3]:
print(nda)


NON-DISCLOSURE AGREEMENT (NDA)

This Agreement is made on September 14, 2023, between ABC Corp., located at 123 Main St., New York, NY, hereinafter referred to as the "Disclosing Party", and XYZ Ltd., located at 456 Elm St., Los Angeles, CA, hereinafter referred to as the "Receiving Party".

Purpose: The Disclosing Party desires to disclose certain confidential information to the Receiving Party for evaluation for a potential business relationship.

Definition of Confidential Information: For purposes of this Agreement, "Confidential Information" shall include all information or material that has or could have commercial value or other utility in the business in which the Disclosing Party is engaged.

Obligations of Receiving Party: The Receiving Party shall hold and maintain the Confidential Information in strictest confidence for the sole and exclusive benefit of the Disclosing Party.

Exclusions from Confidential Information: Receiving Party's obligations under this Agreement do not

In [4]:
match = re.search(r"Exclusions from Confidential Information:(.*?)(?:\n\n|$)", nda, re.DOTALL)
section = match.group(1).strip()
print(section)


Receiving Party's obligations under this Agreement do not extend to information that is: (a) publicly known at the time of disclosure or subsequently becomes publicly known through no fault of the Receiving Party; (b) discovered or created by the Receiving Party before disclosure by Disclosing Party; (c) learned by the Receiving Party through legitimate means other than from the Disclosing Party or Disclosing Party's representatives; or (d) is disclosed by Receiving Party with Disclosing Party's prior written approval.


In [5]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    match = re.search(r"Exclusions from Confidential Information:(.*?)(?:\n\n|$)", text, re.DOTALL)
    section = match.group(1).strip()
    return {"output_text": section}

transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)


In [6]:
template = """Summarize this text:

{output_text}

Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=AzureOpenAI(), prompt=prompt)


In [7]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain],verbose=True)


In [8]:
sequential_chain.run(nda)




> Entering new SimpleSequentialChain chain...
Receiving Party's obligations under this Agreement do not extend to information that is: (a) publicly known at the time of disclosure or subsequently becomes publicly known through no fault of the Receiving Party; (b) discovered or created by the Receiving Party before disclosure by Disclosing Party; (c) learned by the Receiving Party through legitimate means other than from the Disclosing Party or Disclosing Party's representatives; or (d) is disclosed by Receiving Party with Disclosing Party's prior written approval.


InvalidRequestError: Resource not found